In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

from IPython.display import clear_output
from datetime import datetime

In [2]:
param_date = '20240726'

In [3]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel(f'./{param_date}/2.group.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['Manufacturer No'] = input_['Manufacturer No'].astype(int)
input_['Group No'] = input_['Group No'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        import requests
        import json

        import sys
        sys.path.append('../../00.Tools')
        from crawler_configuration import get_header, get_proxy

        data = {'func': 'partsearchgetparttypessdd',
                'mfr': input_.loc[a, 'Manufacturer'],
                'pg': input_.loc[a, 'Group'],
                'giveback': json.dumps({'id': 'partsearch_007'}),
                'api_json_request': '1'}

        try:
            b = 0
            while True:

                while True:
                    b += 1
                    try:
                        resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                             data=data,
                                             headers=get_header(),
                                             proxies=get_proxy(),
                                             timeout=(10, 10))
                        break
                    except:
                        continue

                if resp.status_code == 200:
                    json_ = resp.json()
                    if 'redirect_to_url' in json_:
                        continue
                    elif json_['giveback']['id'] == 'partsearch_007' and 'collected_javascript' in json_:
                        break

            # = = = = = = = = = = = = = = =

            from bs4 import BeautifulSoup
            from lxml import etree

            soup = BeautifulSoup(json_['html_replace_sections']['ddrepl[parttype_][container]'], 'lxml')
            html = etree.HTML(str(soup))

            # = = = = = = = = = = = = = = =

            list_type_code = html.xpath('//select[@id="parttype_"]/option/@value')[1:]

            # = = = = = = = = = = = = = = =

            list_type = html.xpath('//select[@id="parttype_"]/option/text()')[1:]

            # = = = = = = = = = = = = = = =

            if not list_type:
                raise

            # = = = = = = = = = = = = = = =

            df_temp = pd.DataFrame({'Manufacturer No': input_.loc[a, 'Manufacturer No'],
                                    'Manufacturer': input_.loc[a, 'Manufacturer'],
                                    'Group No': input_.loc[a, 'Group No'],
                                    'Group': input_.loc[a, 'Group'],
                                    'Type No': [i+1 for i in range(len(list_type))],
                                    'Type': list_type,
                                    'Type Code': list_type_code,
                                    'Tab 2 Url': [f'''https://www.rockauto.com/en/parts/{input_.loc[a, 'Manufacturer'].replace(',', '%2C').replace('+', '%2B')},{type_.replace(',', '%2C')},{type_code.replace(',', '%2C')}''' for type_, type_code in zip(list_type, list_type_code)]})

            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            if crawler_status == 'error':
                df_temp = pd.DataFrame([{'Manufacturer No': input_.loc[a, 'Manufacturer No'],
                                         'Manufacturer': input_.loc[a, 'Manufacturer'],
                                         'Group No': input_.loc[a, 'Group No'],
                                         'Group': input_.loc[a, 'Group']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b}] - {input_.loc[a, 'Manufacturer No']}.{input_.loc[a, 'Manufacturer']} > {input_.loc[a, 'Group No']}.{input_.loc[a, 'Group']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['Manufacturer No', 'Group', 'Type No'],
                                                ascending=[True, True, True],
                                                ignore_index=True)
    output_correct.to_excel(f'''./{param_date}/3.type-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['Manufacturer No', 'Group'],
                                            ascending=[True, True],
                                            ignore_index=True)
    output_error.to_excel(f'./{param_date}/type_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

[状态：ok，尝试次数：17] - 325.NEEDA > 3.Hardware
[剩余数量：1000] - [当前时间：15:48:34]

[状态：ok，尝试次数：9] - 328.NGK > 1.Exhaust & Emission
[剩余数量：999] - [当前时间：15:48:34]

[状态：ok，尝试次数：45] - 317.MOTORCRAFT > 10.Engine
[剩余数量：998] - [当前时间：15:48:34]

[状态：ok，尝试次数：2] - 330.NISSAN > 3.Cooling System
[剩余数量：997] - [当前时间：15:48:34]

[状态：ok，尝试次数：18] - 329.NICKSON > 1.Exhaust & Emission
[剩余数量：996] - [当前时间：15:48:35]

[状态：ok，尝试次数：5] - 330.NISSAN > 6.Electrical-Switch & Relay
[剩余数量：995] - [当前时间：15:48:35]

[状态：ok，尝试次数：1] - 330.NISSAN > 8.Exhaust & Emission
[剩余数量：994] - [当前时间：15:48:35]

[状态：ok，尝试次数：3] - 330.NISSAN > 11.Heat & Air Conditioning
[剩余数量：993] - [当前时间：15:48:35]

[状态：ok，尝试次数：23] - 330.NISSAN > 2.Brake & Wheel Hub
[剩余数量：992] - [当前时间：15:48:35]

[状态：ok，尝试次数：5] - 330.NISSAN > 1.Body & Lamp Assembly
[剩余数量：991] - [当前时间：15:48:35]

[状态：ok，尝试次数：16] - 328.NGK > 2.Hoses/Lines & Clamps
[剩余数量：990] - [当前时间：15:48:35]

[状态：ok，尝试次数：31] - 323.NATIONAL > 8.Hardware
[剩余数量：989] - [当前时间：15:48:35]

[状态：ok，尝试次数：38] - 321.NAPA/UNITED > 2.Tr